In [4]:
import pandas as pd
import ijson
import json
import os

Trying to extract sample out of 28 GB JSON file by parsing with ijson.

JSON structure:

{"articles": [
	{"abstractText":"text..", "journal":"journal..", "meshMajor":["mesh1",...,"meshN"], "pmid":"PMID", "title":"title..", "year":"YYYY"},
	..., 
	{..}
]}

Wegen ***IncompleteJSONError*** muss File umcodiert werden. Die Funktion liest die Datei in Chunks und versucht, jeden Chunk als UTF-8 zu dekodieren. Ungültige Byte-Sequenzen werden durch das Unicode-Ersatzzeichen ersetzt.

In [11]:
filename = "data/allMeSH_2022/allMeSH_2022.json"

def clean_json_file(input_filename, output_filename):
    total_size = os.path.getsize(input_filename)
    processed_size = 0

    with open(input_filename, 'rb') as input_file, open(output_filename, 'wb') as output_file:
        while True:
            chunk = input_file.read(1024 * 1024)  # Lese die Datei in 1MB Chunks
            if not chunk:
                break  # Beende, wenn das Ende der Datei erreicht ist

            clean_chunk = chunk.decode('utf-8', errors='replace').encode('utf-8')
            output_file.write(clean_chunk)

            processed_size += len(chunk)
            progress = (processed_size / total_size) * 100
            print(f"\rFortschritt: {progress:.2f}%", end='')

    print("\nBereinigung abgeschlossen.")

clean_json_file(filename, 'bereinigte_datei.json')

Fortschritt: 100.00%
Bereinigung abgeschlossen.


Extrahieren eines Sample-Chunks aus der 28 GB JSON Datei

In [24]:
filename = "data/bereinigte_datei.json"
articles = []
max_articles = 1000

with open(filename, 'rb') as input_file:
    # Hier passen wir den Pfad an, um die Artikel direkt zu erreichen
    parser = ijson.items(input_file, 'articles.item')
    for article in parser:
        articles.append(article)
        if len(articles) >= max_articles:
            break

# speichern des samples

# Speichere die extrahierten Artikel in einer neuen JSON-Datei
with open('data/10k_sample_article.json', 'w', encoding='utf-8') as output_file:
    json.dump(articles, output_file, ensure_ascii=False, indent=4)
    
print(f"{len(articles)} Artikel wurden in '10k_sample_article.json' gespeichert.")

1000 Artikel wurden in '10k_sample_article.json' gespeichert.


In [38]:
articles[1]

{'journal': 'Biosensors',
 'meshMajor': ['Cell Separation',
  'Lab-On-A-Chip Devices',
  'Leukocytes',
  'Microfluidic Analytical Techniques',
  'Microfluidics'],
 'year': '2021',
 'abstractText': 'Rapid isolation of white blood cells (WBCs) from whole blood is an essential part of any WBC examination platform. However, most conventional cell separation techniques are labor-intensive and low throughput, require large volumes of samples, need extensive cell manipulation, and have low purity. To address these challenges, we report the design and fabrication of a passive, label-free microfluidic device with a unique U-shaped cross-section to separate WBCs from whole blood using hydrodynamic forces that exist in a microchannel with curvilinear geometry. It is shown that the spiral microchannel with a U-shaped cross-section concentrates larger blood cells (e.g., WBCs) in the inner cross-section of the microchannel by moving smaller blood cells (e.g., RBCs and platelets) to the outer microch

#### MeSH

Durch MeSH sollen Beziehungen bzw. Kanten im Graphen Modelliert werden.

- MeSH (Medical Subject Headings) ist der NLM-kontrollierte Vokabelthesaurus, der zur Indexierung von Artikeln für PubMed verwendet wird.

- Zusätzlich wären Citations von Papers als Kanten interessant. 

Überprüfung ob alle Artikel ein Attribut "meshMajor" mit mindestens 1 Eintrag haben.

In [20]:
# Datei laden, die die extrahierten Artikel enthält
filename = 'data/10k_sample_article.json'

with open(filename, 'r', encoding='utf-8') as file:
    articles = json.load(file)

# Überprüfe jedes Artikelobjekt auf das Vorhandensein und den Inhalt des "meshMajor"-Attributs
missing_or_empty_meshMajor = [article for article in articles if not article.get('meshMajor')]

# Ausgabe der Ergebnisse
if missing_or_empty_meshMajor:
    print(f"{len(missing_or_empty_meshMajor)} von {len(articles)} Artikeln haben kein 'meshMajor'-Attribut oder es ist leer.")
else:
    print("Alle Artikel haben das 'meshMajor'-Attribut mit mindestens einem Eintrag.")

Alle Artikel haben das 'meshMajor'-Attribut mit mindestens einem Eintrag.


Zählen der Artikel, und jener die nicht älter als 2020 sind.

In [6]:
filename = 'data/bereinigte_datei.json'

total_count = 0

with open(filename, 'rb') as file:
    articles = ijson.items(file, 'articles.item')
    for article in articles:
        total_count += 1

print(f"Gesamtanzahl der Artikel: {total_count}")

Gesamtanzahl der Artikel: 16218838


In [5]:
count = 0

with open(filename, 'rb') as file:
    articles = ijson.items(file, 'articles.item')
    for article in articles:
        if 'year' in article and article['year'] and int(article['year']) >= 2020:
            count += 1

print(f"Anzahl der Artikel, die nicht älter als 2020 sind: {count}")

Anzahl der Artikel, die nicht älter als 2020 sind: 862121


In [5]:
filename = "data/bereinigte_datei.json"
articles = []

with open(filename, 'rb') as input_file:
    parser = ijson.items(input_file, 'articles.item')
    for article in parser:  # Iteriere über parser
        if 'year' in article and article['year'] and int(article['year']) >= 2020:
            articles.append(article)

# Speichere die extrahierten Artikel in einer neuen JSON-Datei
with open('data/article_2020.json', 'w', encoding='utf-8') as output_file:
    json.dump(articles, output_file, ensure_ascii=False, indent=4)

print(f"{len(articles)} Artikel wurden in 'data/article_2020.json' gespeichert.")

862121 Artikel wurden in 'data/article_2020.json' gespeichert.


Überprüfung, ob alle es Attribute gibt die keinen Wert enthalten, was zu Fehler beim Datenbankimport führt

In [35]:
# Definiere die erwarteten Attribute jedes Artikels
erwartete_attribute = ["pmid", "title", "abstractText", "year", "journal", "meshMajor"]

# Lade die JSON-Datei
filename = 'data/10k_sample_article.json'
with open(filename, 'r', encoding='utf-8') as file:
    articles = json.load(file)

# Initialisiere ein Dictionary, um die fehlenden Einträge zu zählen
fehlende_eintrage = {attribut: 0 for attribut in erwartete_attribute}

# Überprüfe jedes Attribut in jedem Artikel
for article in articles:
    for attribut in erwartete_attribute:
        if attribut not in article or not article[attribut]:
            fehlende_eintrage[attribut] += 1

# Gib die Anzahl der fehlenden Einträge für jedes Attribut aus
for attribut, anzahl in fehlende_eintrage.items():
    if anzahl > 0:
        print(f"{anzahl} Einträge fehlen im Attribut '{attribut}'.")
    else:
        pass

# Prüfe, ob überhaupt fehlende Einträge existieren
if all(anzahl == 0 for anzahl in fehlende_eintrage.values()):
    print("Alle Artikel haben alle erwarteten Attribute mit Einträgen.")
else:
    print("Einige Artikel haben fehlende oder leere Attribute.")


18 Einträge fehlen im Attribut 'year'.
Einige Artikel haben fehlende oder leere Attribute.


#### First Import in Neo4j

In [22]:
from neo4j import GraphDatabase

In [23]:
uri = "bolt://localhost:7687" 
user = "graph1" 
password = "pubmedgraph" 

driver = GraphDatabase.driver(uri, auth=(user, password))

In [36]:
def import_data(driver, articles):
    with driver.session() as session:
        for article in articles:
            # Prüfe, ob 'year' null ist und ersetze es ggf. durch einen Standardwert oder lasse es weg
            if article['year'] is None:
                print(f"Warnung: Artikel {article['pmid']} hat kein Jahr. Überspringe 'year'.")
                year_to_insert = "Unknown"  # Oder wähle einen anderen passenden Standardwert
            else:
                year_to_insert = article['year']
            
            # Erstelle einen Knoten für den Artikel, wobei 'year' ggf. durch einen Standardwert ersetzt wird
            session.run("MERGE (a:Article {pmid: $pmid, title: $title, abstractText: $abstractText, year: $year, journal: $journal})", 
                        pmid=article['pmid'], title=article['title'], abstractText=article['abstractText'], year=year_to_insert, journal=article['journal'])
            
            # Erstelle Knoten für MeSH-Begriffe und Kanten zu Artikeln
            for mesh in article.get('meshMajor', []):  # Verwende get, um einen leeren Standardwert zu liefern, falls 'meshMajor' nicht existiert
                session.run("MERGE (m:MeSH {term: $term}) MERGE (a:Article {pmid: $pmid}) MERGE (a)-[:TAGGED_WITH]->(m)", term=mesh, pmid=article['pmid'])


# Lese die JSON-Daten ein
with open('data/10k_sample_article.json', 'r', encoding='utf-8') as file:
    articles = json.load(file)

# Importiere die Daten in Neo4j
import_data(driver, articles)

driver.close()

Warnung: Artikel 34818923 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34786969 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34785528 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34785529 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34785531 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34777382 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34761633 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34759044 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34759045 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34759046 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34759047 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34757857 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34755562 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34728520 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34728522 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 34728523 hat kein Jahr. Überspringe 'year'.
Warnung: Artikel 3472852

In [2]:
from web_interface.pycode.chat import Chat

chat = Chat()
user_message = "what is as a novel candidate for treating COVID-19 via heme oxygenase-1 induction."
response = chat.create_chat(user_message)
print(response)

A novel candidate for treating COVID-19 via heme oxygenase-1 induction is heme arginate. This compound has shown potential in studies for inducing heme oxygenase-1, which is believed to have a protective effect against lung injury and inflammation in COVID-19 patients. Heme arginate has been proposed as a therapy to mitigate the severe respiratory symptoms associated with COVID-19 by targeting the heme oxygenase-1 pathway.


In [9]:
filename = 'data/BioASQ-training11b/training11b.json'
qas = []
max_articles = 10

with open(filename, 'rb') as input_file:
    # Hier passen wir den Pfad an, um die Artikel direkt zu erreichen
    parser = ijson.items(input_file, 'questions.item')
    for article in parser:
        qas.append(article)
        if len(qas) >= max_articles:
            break

In [15]:
question = qas[0]['body']
answer = qas[0]['ideal_answer']

In [16]:
response = chat.create_chat(question)
print(response)

Hirschsprung disease is a Mendelian disorder, specifically a genetic disorder caused by mutations in genes related to the development of nerve cells in the intestines. It is typically inherited in an autosomal dominant fashion, meaning that a child only needs to inherit one copy of the mutated gene from one parent to develop the disease.


In [17]:
answer

["Coding sequence mutations in RET, GDNF, EDNRB, EDN3, and SOX10 are involved in the development of Hirschsprung disease. The majority of these genes was shown to be related to Mendelian syndromic forms of Hirschsprung's disease, whereas the non-Mendelian inheritance of sporadic non-syndromic Hirschsprung disease proved to be complex; involvement of multiple loci was demonstrated in a multiplicative model."]